<a href="https://colab.research.google.com/github/keijikk/ml_study/blob/master/01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import tensorflow as tf
from time import gmtime, strftime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import TensorBoard

from google.colab import drive
drive.mount('/content/drive')
tb_home_dir = '/content/drive/My Drive/ML/直感 Deep Learning/drive/'


! npm install -g localtunnel
!rm url.txt

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(tb_home_dir)
)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')


def make_tensorboard(set_dir_name = ''):
  """return tensorboard"""
  tictoc = strftime('%a_%d_%b_%Y_%H_%M_%S', gmtime())
  directory_name = tictoc
  log_dir = set_dir_name +'_' + directory_name
  os.mkdir(log_dir)
  tensorboard = TensorBoard(log_dir=log_dir)
  return tensorboard

tf.test.gpu_device_name()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
updated 1 package in 0.977s


   ╭─────────────────────────────────────╮
   │                                     │
   │   Update available 5.7.1 → 6.4.1    │
   │     Run npm i -g npm to update      │
   │                                     │
   ╰─────────────────────────────────────╯



''

In [10]:
!cat url.txt

your url is: https://grumpy-cobra-3.localtunnel.me


In [0]:
np.random.seed(1671)

NB_EPOCH = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2


In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

RESHAPED = 28*28
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train =X_train.astype('float32')
X_test =X_test.astype('float32')

# normalize
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train_samples')
print(X_test.shape[0], 'test_samples')

# convert class vectors to binary class matrices
print(y_train)
utils.to_categorical(y_train, NB_CLASSES)

Y_train = utils.to_categorical(y_train, NB_CLASSES)
Y_test = utils.to_categorical(y_test, NB_CLASSES)

11493376/11490434 [==============================] - 0s 0us/step
60000 train_samples
10000 test_samples
[5 0 4 ... 5 6 8]


# 1層のニューラルネットワーク

In [5]:
model = Sequential()
model.add(Dense(NB_CLASSES, input_shape=(RESHAPED,), name='Layer_1'))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb = make_tensorboard(set_dir_name=tb_home_dir)
callbacks = [tb]
try:
  model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs=NB_EPOCH, callbacks=callbacks, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
except KeyboardInterrupt:
  model.save(tb.log_dir + '/model.h5')
  callback.writer.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 10)                7850      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 2s 46us/step - loss: 1.3997 - acc: 0.6586 - val_loss: 0.8977 - val_acc: 0.8285
Epoch 2/200
48000/48000 [==============================] - 1s 26us/step - loss: 0.7982 - acc: 0.8241 - val_loss: 0.6568 - val_acc: 0.8569
Epoch 3/200
48000/48000 [==============================] - 1s 25us/step - loss: 0.6455 - acc: 0.8492 - val_loss: 0.5598 - val_acc: 0.8710
Epoch 4/200
48000/48000 [==============================] - 1s 26us/step 

In [6]:
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 0s 19us/step
Test score: 0.2776339444845915
Test accuracy: 0.9216


# 中間層を入れてみる

In [13]:
np.random.seed(1671)

NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2


model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,), name='Layer_1'))
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN, name='Layer_2'))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES, name='Layer_output'))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb = make_tensorboard(set_dir_name=tb_home_dir+"mnist_v2")
callbacks = [tb]

try:
  model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs=NB_EPOCH, callbacks=callbacks, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
except KeyboardInterrupt:
  model.save(tb.log_dir + '/model.h5')
  callback.writer.close()
 
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_7 (Activation)    (None, 128)               0         
_________________________________________________________________
Layer_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
Layer_output (Dense)         (None, 10)                1290      
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Trai

# Dropoutを入れてみる

In [8]:
np.random.seed(1671)

NB_EPOCH = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,), name='Layer_1'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN, name='Layer_2'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES, name='Layer_output'))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb = make_tensorboard(set_dir_name=tb_home_dir+"mnist_v3")
callbacks = [tb]

try:
  model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs=NB_EPOCH, callbacks=callbacks, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
except KeyboardInterrupt:
  model.save(tb.log_dir + '/model.h5')
  callback.writer.close()

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
Layer_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
Layer_output (Dense)         (None, 10)                1290      
__________

# 最適化方法を変えてみる

## RMSprop

In [16]:
from tensorflow.keras.optimizers import RMSprop, Adam

np.random.seed(1671)

NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = RMSprop()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,), name='Layer_1'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN, name='Layer_2'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES, name='Layer_output'))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb = make_tensorboard(set_dir_name=tb_home_dir+"mnist_v3")
callbacks = [tb]

try:
  model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs=NB_EPOCH, callbacks=callbacks, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
except KeyboardInterrupt:
  model.save(tb.log_dir + '/model.h5')
  callback.writer.close()

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_10 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
Layer_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_11 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
Layer_output (Dense)         (None, 10)                1290      
__________

## Adam

In [22]:
np.random.seed(1671)

NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = Adam()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,), name='Layer_1'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN, name='Layer_2'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES, name='Layer_output'))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb = make_tensorboard(set_dir_name=tb_home_dir+"mnist_v3")
callbacks = [tb]

try:
  model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs=NB_EPOCH, callbacks=callbacks, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
except KeyboardInterrupt:
  model.save(tb.log_dir + '/model.h5')
  callback.writer.close()

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_25 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
Layer_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_26 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
Layer_output (Dense)         (None, 10)                1290      
__________

# 正則化

In [21]:
from tensorflow.keras import regularizers

np.random.seed(1671)

NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = Adam()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,),kernel_regularizer=regularizers.l2(0.01), name='Layer_1'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN,kernel_regularizer=regularizers.l2(0.01), name='Layer_2'))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES,kernel_regularizer=regularizers.l2(0.01), name='Layer_output'))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb = make_tensorboard(set_dir_name=tb_home_dir+"mnist_v3")
callbacks = [tb]

try:
  model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs=NB_EPOCH, callbacks=callbacks, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
except KeyboardInterrupt:
  model.save(tb.log_dir + '/model.h5')
  callback.writer.close()

score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_22 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
Layer_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_23 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
Layer_output (Dense)         (None, 10)                1290      
__________